# RTP Update

## Setup

In [3]:
# import packages
import pandas as pd
import pathlib
from pathlib import Path
import os
import arcpy
from utils import *
import numpy as np
import pickle
# external connection packages
from sqlalchemy.engine import URL
from sqlalchemy import create_engine

# pandas options
pd.options.mode.copy_on_write = True
pd.options.mode.chained_assignment = None
pd.options.display.max_columns = 999
pd.options.display.max_rows    = 999

# my workspace 
workspace = r"C:\Users\mbindl\Desktop\Workspace.gdb"
# current working directory
local_path = pathlib.Path().absolute()
# set data path as a subfolder of the current working directory TravelDemandModel\2022\
data_dir = local_path.parents[0] / 'data'
# folder to save processed data
out_dir  = local_path.parents[0] / 'data/processed_data'
# workspace gdb for stuff that doesnt work in memory
# gdb = os.path.join(local_path,'Workspace.gdb')
gdb = workspace
# set environement workspace to in memory 
arcpy.env.workspace = 'memory'
# # clear memory workspace
# arcpy.management.Delete('memory')

# overwrite true
arcpy.env.overwriteOutput = True
# Set spatial reference to NAD 1983 UTM Zone 10N
sr = arcpy.SpatialReference(26910)

# get parcels from the database
# network path to connection files
filePath = "F:/GIS/PARCELUPDATE/Workspace/"
# database file path 
sdeBase    = os.path.join(filePath, "Vector.sde")
sdeCollect = os.path.join(filePath, "Collection.sde")
sdeTabular = os.path.join(filePath, "Tabular.sde")
sdeEdit    = os.path.join(filePath, "Edit.sde")

# Pickle variables
# part 1 - spatial join categories, occupancy rates, and parcels
parcel_pickle_part1    = data_dir / 'parcel_pickle1.pkl'

# columsn to list
initial_columns = [ 'APN',
                    'APO_ADDRESS',
                    'Residential_Units',
                    'TouristAccommodation_Units',
                    'CommercialFloorArea_SqFt',
                    'YEAR',
                    'JURISDICTION',
                    'COUNTY',
                    'OWNERSHIP_TYPE',
                    'COUNTY_LANDUSE_DESCRIPTION',
                    'EXISTING_LANDUSE',
                    'REGIONAL_LANDUSE',
                    'YEAR_BUILT',
                    'PLAN_ID',
                    'PLAN_NAME',
                    'ZONING_ID',
                    'ZONING_DESCRIPTION',
                    'TOWN_CENTER',
                    'LOCATION_TO_TOWNCENTER',
                    'TAZ',
                    'PARCEL_ACRES',
                    'PARCEL_SQFT',
                    'WITHIN_BONUSUNIT_BNDY',
                    'WITHIN_TRPA_BNDY',
                    'SHAPE']

# schema for the final output
final_schema = [ 'APN',
                'APO_ADDRESS',
                'Residential_Units',
                'TouristAccommodation_Units',
                'CommercialFloorArea_SqFt',
                'YEAR',
                'JURISDICTION',
                'COUNTY',
                'OWNERSHIP_TYPE',
                'COUNTY_LANDUSE_DESCRIPTION',
                'EXISTING_LANDUSE',
                'REGIONAL_LANDUSE',
                'YEAR_BUILT',
                'PLAN_ID',
                'PLAN_NAME',
                'ZONING_ID',
                'ZONING_DESCRIPTION',
                'TOWN_CENTER',
                'LOCATION_TO_TOWNCENTER',
                'TAZ',
                'PARCEL_ACRES',
                'PARCEL_SQFT',
                'WITHIN_BONUSUNIT_BNDY',
                'WITHIN_TRPA_BNDY',
                'IPES_SCORE',
                'IPES_SCORE_TYPE',
                'RETIRED',
                'HOUSING_ZONING',
                'MAX_RESIDENTIAL_UNITS', 
                'MAX_COMMERCIAL_FLOOR_AREA', 
                'MAX_TAU_UNITS',
                'SHAPE']

In [67]:
def get_conn(db):
    # Get database user and password from environment variables on machine running script
    db_user             = os.environ.get('DB_USER')
    db_password         = os.environ.get('DB_PASSWORD')
    # driver is the ODBC driver for SQL Server
    driver              = 'ODBC Driver 17 for SQL Server'
    # server names are
    sql_12              = 'sql12'
    sql_14              = 'sql14'
    # make it case insensitive
    db = db.lower()
    # make sql database connection with pyodbc
    if db   == 'sde_tabular':
        connection_string = f"DRIVER={driver};SERVER={sql_12};DATABASE={db};UID={db_user};PWD={db_password}"
        connection_url = URL.create("mssql+pyodbc", query={"odbc_connect": connection_string})
        engine = create_engine(connection_url)
    elif db == 'tahoebmpsde':
        connection_string = f"DRIVER={driver};SERVER={sql_14};DATABASE={db};UID={db_user};PWD={db_password}"
        connection_url = URL.create("mssql+pyodbc", query={"odbc_connect": connection_string})
        engine = create_engine(connection_url)
    elif db == 'sde':
        connection_string = f"DRIVER={driver};SERVER={sql_12};DATABASE={db};UID={db_user};PWD={db_password}"
        connection_url = URL.create("mssql+pyodbc", query={"odbc_connect": connection_string})
        engine = create_engine(connection_url)
    # else return None
    else:
        engine = None
    # connection file to use in pd.read_sql
    return engine

# save to pickle
def to_pickle(data, filename):
    with open(filename, 'wb') as f:
        pickle.dump(data, f)
    print(f'{filename} pickled')

# save to pickle and feature class
def to_pickle_fc(data, filename):
    data.spatial.to_featureclass(filename)
    with open(filename, 'wb') as f:
        pickle.dump(data, f)
    print(f'{filename} pickled and saved as feature class')

# get a pickled file as a dataframe
def from_pickle(filename):
    with open(filename, 'rb') as f:
        data = pickle.load(f)
    print(f'{filename} unpickled')
    return data

# function to get where Zoningin_ID Use_Type = Multi-Family and Density
def get_mf_zones(df):
    columns_to_keep = ['Zoning_ID', 'Use_Type', 'Density']
    # filter Use_Type to Multiple Family Dwelling
    df = df.loc[df['Use_Type'] == 'Multiple Family Dwelling']
    return df[columns_to_keep]

def get_sf_zones(df):
    columns_to_keep = ['Zoning_ID', 'Use_Type', 'Density']
    # filter Use_Type to Single Family Dwelling
    df = df.loc[df['Use_Type'] == 'Single Family Dwelling']
    return df[columns_to_keep]

def get_sf_only_zones(df):
    columns_to_keep = ['Zoning_ID', 'Use_Type', 'Density']
    # filter Use_Type to Single Family Dwelling and not Multiple Family Dwelling
    dfMF = get_mf_zones(df)
    dfSF = get_sf_zones(df)
    df = dfSF.loc[~dfSF['Zoning_ID'].isin(dfMF['Zoning_ID'])]
    return df[columns_to_keep]

def get_mf_only_zones(df):
    columns_to_keep = ['Zoning_ID', 'Use_Type', 'Density']
    # filter Use_Type to Multiple Family Dwelling and not Single Family Dwelling
    dfSF = get_sf_zones(df)
    dfMF = get_mf_zones(df)
    df = dfMF.loc[~dfMF['Zoning_ID'].isin(dfSF['Zoning_ID'])]
    return df[columns_to_keep]

def get_recieving_zones(df):
    columns_to_keep = ['Zoning_ID', 'SPECIAL_DESIGNATION']
    # filter transfer recieving
    df = df.loc[df['SPECIAL_DESIGNATION'] == 'Receive']
    return df[columns_to_keep]

def get_sending_zones(df):
    columns_to_keep = ['Zoning_ID', 'SPECIAL_DESIGNATION']
    df = df.loc[df['SPECIAL_DESIGNATION'] == 'Transfer']
    return df[columns_to_keep]


### Get Data

In [ ]:
# parcel development layer polygons
parcel_db = Path(sdeEdit) / "SDE.Parcel\\SDE.Parcel_History_Attributed"
# query 2022 rows
sdf_units = pd.DataFrame.spatial.from_featureclass(parcel_db)
sdf_units = sdf_units.loc[sdf_units['YEAR'] == 2022]
sdf_units.spatial.sr = sr

# # get parcel level data from Collection SDE
# vhr feature layer polygons 
vhr_db = Path(sdeCollect) / "SDE.Parcel\\SDE.Parcel_VHR"
sdf_vhr = pd.DataFrame.spatial.from_featureclass(vhr_db)
sdf_vhr.spatial.sr = sr
# filter vhr layer to active status
sdf_vhr = sdf_vhr.loc[sdf_vhr['Status'] == 'Active']

# TAZ feature layer polygons
taz_db = Path(sdeBase) / "SDE.Transportation\\SDE.Transportation_Analysis_Zone"
# get as spatial dataframe
sdf_taz = pd.DataFrame.spatial.from_featureclass(taz_db)
# set spatial reference to NAD 1983 UTM Zone 10N
sdf_taz.spatial.sr = sr

# censuse feature class
census_fc    = Path(sdeBase) / "SDE.Census\\SDE.Tahoe_Census_Geography"
# bouns unit boundary feature class
bonus_unit_fc = Path(sdeBase) / "SDE.Planning\SDE.Bonus_unit_boundary"

# disable Z values on block group feature layer
with arcpy.EnvManager(outputZFlag="Disabled"):    
    arcpy.conversion.FeatureClassToGeodatabase(
        Input_Features="F:\GIS\DB_CONNECT\Vector.sde\SDE.Census\SDE.Tahoe_Census_Geography",
        Output_Geodatabase=r"C:\Users\mbindl\Desktop\Workspace.gdb"
    )
# disable Z values on block group feature layer
with arcpy.EnvManager(outputZFlag="Disabled"):    
    arcpy.conversion.FeatureClassToGeodatabase(
        Input_Features="F:\GIS\DB_CONNECT\Vector.sde\SDE.Planning\SDE.Bonus_unit_boundary",
        Output_Geodatabase=r"C:\Users\mbindl\Desktop\Workspace.gdb"
    )

# block group feature layer polygons with no Z
sdf_block = pd.DataFrame.spatial.from_featureclass(Path(gdb) / 'Tahoe_Census_Geography')
sdf_block = sdf_block.loc[(sdf_block['YEAR'] == 2020) & (sdf_block['GEOGRAPHY'] == 'Block Group')]
sdf_block.spatial.sr = sr

# bonus unit boundary wihtout Z
sdf_bonus = pd.DataFrame.spatial.from_featureclass(Path(gdb) / 'Bonus_unit_boundary')
sdf_bonus.spatial.sr = sr

# get parcel level data from LTinfo
dfIPES       = pd.read_json("https://www.laketahoeinfo.org/WebServices/GetParcelIPESScores/JSON/e17aeb86-85e3-4260-83fd-a2b32501c476")
dfLCV_LTinfo = pd.read_json('https://www.laketahoeinfo.org/WebServices/GetParcelsByLandCapability/JSON/e17aeb86-85e3-4260-83fd-a2b32501c476')
dfRetired    = pd.read_json("https://www.laketahoeinfo.org/WebServices/GetAllParcels/JSON/e17aeb86-85e3-4260-83fd-a2b32501c476")
dfBankedDev  = pd.read_json('https://www.laketahoeinfo.org/WebServices/GetBankedDevelopmentRights/JSON/e17aeb86-85e3-4260-83fd-a2b32501c476')
dfTransacted = pd.read_json('https://www.laketahoeinfo.org/WebServices/GetTransactedAndBankedDevelopmentRights/JSON/e17aeb86-85e3-4260-83fd-a2b32501c476')
dfAllParcels = pd.read_json('https://www.laketahoeinfo.org/WebServices/GetAllParcels/JSON/e17aeb86-85e3-4260-83fd-a2b32501c476')

# get use tables 
# zoning data
sde_engine = get_conn('sde')
with sde_engine.begin() as conn:
    df_uses    = pd.read_sql("SELECT * FROM sde.SDE.PermissibleUses", conn)
    df_special = pd.read_sql("SELECT * FROM sde.SDE.Special_Designation", conn)

## Parcel Data Engineering

In [ ]:
# spatial join to get TAZ
arcpy.SpatialJoin_analysis(sdf_units, sdf_taz, "Existing_Development_TAZ", 
                           "JOIN_ONE_TO_ONE", "KEEP_ALL", "", "HAVE_THEIR_CENTER_IN")
# spatial join to get Block Group
arcpy.SpatialJoin_analysis(sdf_units, sdf_block, "Existing_Development_BlockGroup", 
                           "JOIN_ONE_TO_ONE", "KEEP_ALL", "", "HAVE_THEIR_CENTER_IN")
# spatial join of Bonus Unit Boundary
arcpy.SpatialJoin_analysis(sdf_units, sdf_bonus, "Existing_Development_BonusUnitBoundary",
                            "JOIN_ONE_TO_ONE", "KEEP_ALL", "", "INTERSECT")

In [ ]:
# spatial dataframe with only initial columns
sdfParcels = sdf_units[initial_columns]

# get results of spatial joins as spatial dataframes
sdf_units_taz   = pd.DataFrame.spatial.from_featureclass("Existing_Development_TAZ", sr=sr)  
sdf_units_block = pd.DataFrame.spatial.from_featureclass("Existing_Development_BlockGroup", sr=sr)
sdf_units_bonus = pd.DataFrame.spatial.from_featureclass("Existing_Development_BonusUnitBoundary", sr=sr)
# cast to string
sdf_units_bonus['WITHIN_BONUSUNIT_BNDY'] = sdf_units_bonus['WITHIN_BONUSUNIT_BNDY'].astype(str)
sdf_units_bonus['WITHIN_BONUSUNIT_BNDY'] = 'No'
# if Id is not NA then within bonus unit boundary = yes, else
sdf_units_bonus.loc[sdf_units_bonus['Id'].notna(), 'WITHIN_BONUSUNIT_BNDY'] = 'Yes'

# map dictionary to sdf_units dataframe to fill in TAZ and Block Group fields
sdfParcels['TAZ']                   = sdfParcels.APN.map(dict(zip(sdf_units_taz.APN,   sdf_units_taz.TAZ_1)))
sdfParcels['BLOCK_GROUP']           = sdfParcels.APN.map(dict(zip(sdf_units_block.APN, sdf_units_block.TRPAID)))
# map IPES score to parcels
sdfParcels['IPES_SCORE']            = sdfParcels['APN'].map(dict(zip(dfIPES.APN, dfIPES.IPESScore)))
sdfParcels['IPES_SCORE_TYPE']       = sdfParcels['APN'].map(dict(zip(dfIPES.APN, dfIPES.IPESScoreType)))
# retired parcels
sdfParcels['RETIRED']               = sdfParcels['APN'].map(dict(zip(dfAllParcels.APN, dfAllParcels.RetiredFromDevelopment)))
sdfParcels['WITHIN_BONUSUNIT_BNDY'] = sdfParcels['APN'].map(dict(zip(sdf_units_bonus.APN, sdf_units_bonus.WITHIN_BONUSUNIT_BNDY)))
# define housnig zoning and density
sdfParcels['HOUSING_ZONING']        = 'NA'

# if the zoning id is in the list of single family zones then set to SF
sdfParcels.loc[sdfParcels['ZONING_ID'].isin(get_sf_zones(df_uses)['Zoning_ID']), 'HOUSING_ZONING'] = 'SF'
# if the zoning id is in the list of multiple family zones then set to MF
sdfParcels.loc[sdfParcels['ZONING_ID'].isin(get_mf_zones(df_uses)['Zoning_ID']), 'HOUSING_ZONING'] = 'MF'
# if the zoning id is in the list of single family zones and not in the multiple family zones then set to SF only
sdfParcels.loc[sdfParcels['ZONING_ID'].isin(get_sf_only_zones(df_uses)['Zoning_ID']), 'HOUSING_ZONING'] = 'SF_only'
# if the zoning id is in the list of multiple family zones and not in the single family zones then set to MF only
sdfParcels.loc[sdfParcels['ZONING_ID'].isin(get_mf_only_zones(df_uses)['Zoning_ID']), 'HOUSING_ZONING'] = 'MF_only'

# get density for MF and MF only zones, max residential units, and adjusted residential units
dfMF = get_mf_zones(df_uses)
sdfParcels['DENSITY']                    = sdfParcels['ZONING_ID'].map(dict(zip(dfMF.Zoning_ID, dfMF.Density)))
sdfParcels['MAX_RESIDENTIAL_UNITS']      = sdfParcels['PARCEL_ACRES'] * sdfParcels['DENSITY']
sdfParcels['ADJUSTED_RESIDENTIAL_UNITS'] = sdfParcels['MAX_RESIDENTIAL_UNITS']*0.6

# if COUNTY is in EL or PL and SF allowed then set ADU_ALLOWED to yes
sdfParcels['ADU_ALLOWED'] = 'No'
sdfParcels.loc[(sdfParcels['COUNTY'].isin(['EL','PL'])) & (sdfParcels['HOUSING_ZONING'].isin(['SF', 'SF_only'])), 'ADU_ALLOWED'] = 'Yes'

# export to pickle
sdfParcels.to_pickle(parcel_pickle_part1)
# spatial data frame to feature class
sdfParcels.spatial.to_featureclass('Parcel_Base_2022')

In [ ]:
sdfParcels.ADU_ALLOWED.value_counts()

### Model Years

In [ ]:
tau_assigned_lookup = "Lookup_Lists\forecast_tourist_assigned_units.csv"
tau_zoned_lookup    = "Lookup_Lists\forecast_tourist_zoned_units.csv"
res_assigned_lookup = "Lookup_Lists\forecast_residential_assigned_units.csv"
res_zoned_lookup    = "Lookup_Lists\forecast_residential_zoned_units.csv"
com_assigned_lookup = "Lookup_Lists\forecast_commercial_assigned_units.csv"
com_zoned_lookup    = "Lookup_Lists\forecast_commercial_zoned_units.csv"

In [ ]:
# filter parcels so only APNs in the lookup are included
sdfTAU = sdfParcels[sdfParcels['APN'].isin(tau_assigned_lookup['APN'])]
# get TAU_Type from lookup
sdfParcels['Forecast_Tourist_Units'] = sdfTAU['APN'].map(tau_assigned_lookup.set_index('APN')['TAU_Type'])
# filter parcels so only APNs in the lookup are included
sdfRes = sdfParcels[sdfParcels['APN'].isin(res_assigned_lookup['APN'])]
# set residential units to assigned total
sdfParcels['Forecast_Residential_Units'] = sdfRes['APN'].map(res_assigned_lookup.set_index('APN')['Units'])
# filter parcels so only APNs in the lookup are included
sdfCFA = sdfParcels[sdfParcels['APN'].isin(com_assigned_lookup['APN'])]
# set commercial floor area to assigned total
sdfParcels['Forecast_CommercialFloorArea_SqFt'] = sdfCFA['APN'].map(com_assigned_lookup.set_index('APN')['SqFt'])


In [ ]:
# set forecast year
sdfParcels['Forecast_Year'] = 2035
